In [1]:
%matplotlib inline
import sys
sys.path.append("/home/diego/Dropbox/hold_noise/iclamp-glm/")

import matplotlib.pyplot as plt
import numpy as np

from icglm.models.glm import GLM
from icglm.kernels import KernelRect
from icglm.processes import OUProcess

%load_ext autoreload
%autoreload 2

### Define stimulus process, GLM and sample a spike train

In [2]:
t = np.arange(0, 100000, 1)
mu, sd = 25, 25
ou = OUProcess(mu=mu, sd=sd, tau=3)
stim = ou.sample(t, seed=0)

u0 = 16
kappa = KernelRect.exponential(tf=240, dt=8, tau=40, A=1e-2)
eta = KernelRect.exponential(tf=450, dt=25, tau=75, A=7)
glm_true = GLM(u0=u0, kappa=kappa, eta=eta)

np.random.seed(0)
v, r, mask_spk_true = glm_true.sample(t, stim)

### Fit GLM

In [3]:
newton_kwargs = dict(max_iterations=30, stop_cond=2e-4, learning_rate=5e-1, initial_learning_rate=5e-4, warm_up_iterations=3)

glm = glm_true.copy()
optimizer, logl = glm.fit(t, stim, mask_spk_true, newton_kwargs=newton_kwargs,verbose=True)

fig, (ax1, ax2, ax3) = plt.subplots(figsize=(15, 5), ncols=3)
glm_true.kappa.plot(ax=ax1)
glm.kappa.plot(ax=ax1)
glm_true.eta.plot(ax=ax2)
glm.eta.plot(ax=ax2)
ax3.plot(optimizer.log_posterior_iterations, '-o')

Starting gradient ascent... 

 Iteration 8 of 30 | Elapsed time: 1.11 seconds | log_prior=nan | log_posterior=-2132.16
 
 Iteration 8 of 30 | Converged | Elapsed time: 0.0186 minutes | Log posterior is monotonic 



TypeError: cannot unpack non-iterable NewtonMethod object

### Decode stimulus

In [ ]:
newton_kwargs = dict(max_iterations=30, stop_cond=5e-6, learning_rate=5e-1, initial_learning_rate=5e-2, warm_up_iterations=3)

# ou.set_dt(1)
stim0 = np.zeros(len(t)) + mu
stim_dec, optimizer = glm_true.decode(t, mask_spk_true, stim0=stim0, prior=ou, newton_kwargs=newton_kwargs, verbose=True)

fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(t, stim)
ax.plot(t, stim_dec)

fig, (ax1, ax2) = plt.subplots(figsize=(10, 5), ncols=2)
ax1.plot(optimizer.log_prior_iterations)
ax2.plot(optimizer.log_posterior_iterations, '-o')